**References**:
- The debiasing algorithm is from Bolukbasi et al., 2016, [Man is to Computer Programmer as Woman is to
Homemaker? Debiasing Word Embeddings](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf)


# Operations on word vectors



- Load pre-trained word vectors, and measure similarity using cosine similarity
- Use word embeddings to solve word analogy problems.  
- Modify word embeddings to reduce their gender bias 



In [ ]:
import numpy as np
from w2v_utils import *

#### Load the word vectors
*  Used 50-dimensional GloVe vectors to represent words.  

In [ ]:
words, word_to_vec_map = read_glove_vecs('/content/glove.6B.50d.txt')

- `words`: set of words in the vocabulary.
- `word_to_vec_map`: dictionary mapping words to their GloVe vector representation.
 

# 1 - Cosine similarity

To measure the similarity between two words, we need a way to measure the degree of similarity between two embedding vectors for the two words. Given two vectors $u$ and $v$, cosine similarity is defined as follows: 

$$\text{CosineSimilarity(u, v)} = \frac {u \cdot v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

* $u \cdot v$ is the dot product (or inner product) of two vectors
* $||u||_2$ is the norm (or length) of the vector $u$
* $\theta$ is the angle between $u$ and $v$. 
* The cosine similarity depends on the angle between $u$ and $v$. 
    * If $u$ and $v$ are very similar, their cosine similarity will be close to 1.
    * If they are dissimilar, the cosine similarity will take a smaller value. 

<img src="images/cosine_sim.png" style="width:800px;height:250px;">
<caption><center> **Figure 1**: The cosine of the angle between two vectors is a measure their similarity</center></caption>

In [ ]:
def cosine_similarity(u, v): 
    distance = 0.0 
    # Dot product between u and v  
    dot = np.dot(u,v)
    #  L2 norm of u  
    norm_u =np.sqrt(np.sum(np.dot(u,u))) 
    #  L2 norm of v  
    norm_v =np.sqrt(np.sum(np.dot(v,v)))
    # Cosine similarity  
    cosine_similarity =dot/np.dot(norm_u,norm_v)
    return cosine_similarity

In [ ]:
father = word_to_vec_map["father"]
mother = word_to_vec_map["mother"]
ball = word_to_vec_map["ball"]
crocodile = word_to_vec_map["crocodile"]
france = word_to_vec_map["france"]
italy = word_to_vec_map["italy"]
paris = word_to_vec_map["paris"]
rome = word_to_vec_map["rome"]

print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))
print("cosine_similarity(ball, crocodile) = ",cosine_similarity(ball, crocodile))
print("cosine_similarity(france - paris, rome - italy) = ",cosine_similarity(france - paris, rome - italy))

In [ ]:
man = word_to_vec_map["man"]
woman = word_to_vec_map["woman"]
king = word_to_vec_map["king"]
queen = word_to_vec_map["queen"]
print("cosine_similarity(man - woman, king - queen) = ",cosine_similarity(man - woman, king - queen))
print("cosine_similarity(man , woman) = ",cosine_similarity(man , woman))
print("cosine_similarity(man , king) = ",cosine_similarity(man , king))
print("cosine_similarity(man , queen) = ",cosine_similarity(man , queen))

#### Try different words!


## Word analogy 

* In the word analogy task, we complete the sentence:  
    <font color='brown'>"*a* is to *b* as *c* is to **____**"</font>. 

* An example is:  
    <font color='brown'> '*man* is to *woman* as *king* is to *queen*' </font>. 

* We are trying to find a word *d*, such that the associated word vectors $e_a, e_b, e_c, e_d$ are related in the following manner:   
    $e_b - e_a \approx e_d - e_c$
* We   measure the similarity between $e_b - e_a$ and $e_d - e_c$ using cosine similarity.  

In [ ]:
def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """ 
    word_a -- a word, string
    word_b -- a word, string
    word_c -- a word, string
    word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    # convert words to lowercase
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    # Get the word embeddings e_a, e_b and e_c
    e_a, e_b, e_c = word_to_vec_map[word_a],word_to_vec_map[word_b],word_to_vec_map[word_c]
  
    words = word_to_vec_map.keys()
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_word = None                   # Initialize best_word with None, it will help keep track of the word to output

    # to avoid best_word being one of the input words, skip the input words
    # place the input words in a set for faster searching than a list
    # We  re-use this set of input words inside the for-loop
    input_words_set = set([word_a, word_b, word_c])
    
    # loop over the whole word vector set
    for w in words:        
        # to avoid best_word being one of the input words, skip the input words
        if w in [word_a, word_b, word_c]:
            continue 
        # Compute cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)   
        cosine_sim = cosine_similarity(e_b-e_a, word_to_vec_map[w]-e_c)
        
        # If the cosine_sim is more than the max_cosine_sim seen so far,
            # then: set the new max_cosine_sim to the current cosine_sim and the best_word to the current word 
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w 
    return best_word

In [ ]:
triads_to_try = [('small', 'smaller', 'big'),('man', 'doctor', 'women'),('italy', 'italian', 'spain'), ('india', 'delhi', 'japan'), ('man', 'woman', 'king'), ('small', 'smaller', 'large')]
for triad in triads_to_try:
    print ('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad,word_to_vec_map)))

# Debiasing word vectors 

In this i examine gender biases that can be reflected in a word embedding, and explore algorithms for reducing the bias. 
First see how the GloVe word embeddings relate to gender. First compute a vector $g = e_{woman}-e_{man}$, where $e_{woman}$ represents the word vector corresponding to the word *woman*, and $e_{man}$ corresponds to the word vector corresponding to the word *man*. The resulting vector $g$ roughly encodes the concept of "gender". 

In [ ]:
g = word_to_vec_map['woman'] - word_to_vec_map['man']
print(g)

In [ ]:
print ('List of names and their similarities with constructed vector:')
name_list = ['john', 'marie', 'sohan', 'don', 'danielle', 'reza', 'mohan', 'katy', 'yasmin']

for w in name_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

See, female first names tend to have a positive cosine similarity with our constructed vector $g$, while male first names tend to have a negative cosine similarity. This is not surprising, and the result seems acceptable. 

In [ ]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

(https://arxiv.org/abs/1607.06520)
### Neutralize bias for non-gender specific words 

The figure below should help you visualize what neutralizing does. If you're using a 50-dimensional word embedding, the 50 dimensional space can be split into two parts: The bias-direction $g$, and the remaining 49 dimensions, which we'll call $g_{\perp}$. In linear algebra, we say that the 49 dimensional $g_{\perp}$ is perpendicular (or "orthogonal") to $g$, meaning it is at 90 degrees to $g$. The neutralization step takes a vector such as $e_{receptionist}$ and zeros out the component in the direction of $g$, giving us $e_{receptionist}^{debiased}$. 

Even though $g_{\perp}$ is 49 dimensional, given the limitations of what we can draw on a 2D screen, we illustrate it using a 1 dimensional axis below. 

<img src="images/neutral.png" style="width:800px;height:300px;">
<caption><center> **Figure 2**: The word vector for "receptionist" represented before and after applying the neutralize operation. </center></caption>

**Exercise**: Implement `neutralize()` to remove the bias of words such as "receptionist" or "scientist". Given an input embedding $e$, you can use the following formulas to compute $e^{debiased}$: 

$$e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g\tag{2}$$
$$e^{debiased} = e - e^{bias\_component}\tag{3}$$

  $e^{bias\_component}$ is the projection of $e$ onto the direction $g$.  

<!-- 
**Reminder**: a vector $u$ can be split into two parts: its projection over a vector-axis $v_B$ and its projection over the axis orthogonal to $v$:
$$u = u_B + u_{\perp}$$
where : $u_B = $ and $ u_{\perp} = u - u_B $
!--> 

In [ ]:
def neutralize(word, g, word_to_vec_map):
    """  
        word -- string indicating the word to debias
        g -- numpy-array of shape (50,), corresponding to the bias axis (such as gender)
        word_to_vec_map -- dictionary mapping words to their corresponding vectors.
     
        e_debiased -- neutralized word vector representation of the input "word"
    """
    # Select word vector representation of "word". Use word_to_vec_map.  
    e = word_to_vec_map[word]
    
    # Compute e_biascomponent using the formula give above.  
    e_biascomponent = np.divide(np.dot(e,g),np.linalg.norm(g)**2) * g
 
    # Neutralize e by subtracting e_biascomponent from it 
    # e_debiased should be equal to its orthogonal projection.
    e_debiased = e - e_biascomponent

    return e_debiased

In [ ]:
e = "receptionist"
print("cosine similarity between " + e + " and g, before neutralizing: ", cosine_similarity(word_to_vec_map["receptionist"], g))

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print("cosine similarity between " + e + " and g, after neutralizing: ", cosine_similarity(e_debiased, g))

### 3.2 - Equalization algorithm for gender-specific words

See how debiasing can also be applied to word pairs such as "actress" and "actor." Equalization is applied to pairs of words that you might want to have differ only through the gender property. As a concrete example, suppose that "actress" is closer to "babysit" than "actor." By applying neutralizing to "babysit" we can reduce the gender-stereotype associated with babysitting. But this still does not guarantee that "actor" and "actress" are equidistant from "babysit." The equalization algorithm takes care of this. 

The key idea behind equalization is to make sure that a particular pair of words are equi-distant from the 49-dimensional $g_\perp$. The equalization step also ensures that the two equalized steps are now the same distance from $e_{receptionist}^{debiased}$, or from any other work that has been neutralized. In pictures, this is how equalization works: 

<img src="images/equalize10.png" style="width:800px;height:400px;">


The derivation of the linear algebra to do this is a bit more complex. (See Bolukbasi et al., 2016 for details.) But the key equations are: 

$$ \mu = \frac{e_{w1} + e_{w2}}{2}\tag{4}$$ 

$$ \mu_{B} = \frac {\mu \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{5}$$ 

$$\mu_{\perp} = \mu - \mu_{B} \tag{6}$$

$$ e_{w1B} = \frac {e_{w1} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{7}$$ 
$$ e_{w2B} = \frac {e_{w2} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{8}$$


$$e_{w1B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w1B}} - \mu_B} {||(e_{w1} - \mu_{\perp}) - \mu_B||} \tag{9}$$


$$e_{w2B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w2B}} - \mu_B} {||(e_{w2} - \mu_{\perp}) - \mu_B||} \tag{10}$$

$$e_1 = e_{w1B}^{corrected} + \mu_{\perp} \tag{11}$$
$$e_2 = e_{w2B}^{corrected} + \mu_{\perp} \tag{12}$$


In [ ]:
def equalize(pair, bias_axis, word_to_vec_map):
    """ 
    pair = pair of strings of gender specific words to debias, e.g. ("actress", "actor") 
    bias_axis = numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    word_to_vec_map = dictionary mapping words to their corresponding vectors
   
    e_1 -- word vector corresponding to the first word
    e_2 -- word vector corresponding to the second word
    """
    # Select word vector representation of "word". Use word_to_vec_map.  
    w1, w2 = pair[0], pair[1]
    e_w1, e_w2 = word_to_vec_map[w1], word_to_vec_map[w2]
    
    #  Compute the mean of e_w1 and e_w2  
    mu = (e_w1 + e_w2)/2

    #Compute the projections of mu over the bias axis and the orthogonal axis  
    mu_B = (np.dot(mu, bias_axis)/np.linalg.norm(bias_axis)**2)*bias_axis
    mu_orth = mu - mu_B

    # Use equations (7) and (8) to compute e_w1B and e_w2B  
    e_w1B = (np.dot(e_w1, bias_axis)*np.linalg.norm(bias_axis)**2)*bias_axis
    e_w2B = (np.dot(e_w2, bias_axis)*np.linalg.norm(bias_axis)**2)*bias_axis
        
    #  Adjust the Bias part of e_w1B and e_w2B using the formulas (9) and (10) given above  
    corrected_e_w1B = np.sqrt(1 - np.linalg.norm(mu_orth)**2)*((e_w1B - mu_B)/np.abs((e_w1 - mu_orth) - mu_B))
    corrected_e_w2B = np.sqrt(1 - np.linalg.norm(mu_orth)**2)*((e_w2B - mu_B)/np.abs((e_w2 - mu_orth) - mu_B))

    # Debias by equalizing e1 and e2 to the sum of their corrected projections 
    e1 = corrected_e_w1B + mu_orth
    e2 = corrected_e_w2B + mu_orth
    return e1, e2

In [ ]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], g))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], g))
print()
e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))